In [86]:
import pandas as pd

In [87]:
id = '1alaeZRchzrXYqNJ_WGEk8DkcjRkIOn-BXtkCqqGMXBM'
sheet_url = f'https://docs.google.com/spreadsheets/d/{id}/export?format=csv'

tab_id_g = '0'
tab_id_c = '743163027'
tab_id_u = '897089842'

url_ganhadas = f'{sheet_url}&gid={tab_id_g}'
url_compradas = f'{sheet_url}&gid={tab_id_c}'
url_utilizadas = f'{sheet_url}&gid={tab_id_u}'

df_ganhadas = pd.read_csv(url_ganhadas)
df_compradas = pd.read_csv(url_compradas)
df_utilizadas = pd.read_csv(url_utilizadas)

In [88]:
df_ganhadas.head(3)

,marca,submarca,quantidade,tamanho,roupinha,data,evento
0,Huggies,Supreme Care,30,G,S,14/09/2024,Chá de Fraldas amigos
1,Huggies,Supreme Care,30,G,S,14/09/2024,Chá de Fraldas amigos
2,Huggies,Supreme Care,30,G,S,14/09/2024,Chá de Fraldas amigos


In [89]:
df_compradas.head(3)

,marca,submarca,quantidade,tamanho,roupinha,data,valor
0,Huggies,Supreme Care,30,G,S,20/09/2024,"45,9"


In [90]:
df_utilizadas.head(3)

,marca,submarca,quantidade,tamanho,roupinha,data
0,Huggies,Supreme Care,6,G,S,20/09/2024


In [91]:
df_estoque = pd.concat([df_compradas, df_ganhadas], ignore_index=True).drop(columns=['valor', 'evento'])
df_estoque.head(3)

,marca,submarca,quantidade,tamanho,roupinha,data
0,Huggies,Supreme Care,30,G,S,20/09/2024
1,Huggies,Supreme Care,30,G,S,14/09/2024
2,Huggies,Supreme Care,30,G,S,14/09/2024


In [92]:
int(df_estoque[df_estoque['tamanho'] == 'M']['quantidade'].sum())

956

In [93]:
df_estoque_gp = pd.DataFrame(df_estoque.groupby(['tamanho'])['quantidade'].sum()).reset_index()
df_estoque_gp

,tamanho,quantidade
0,G,246
1,M,956
2,P,272
3,RN+,56


In [94]:
df_utilizadas_gp = pd.DataFrame(df_utilizadas.groupby(['tamanho'])['quantidade'].sum()).reset_index()
df_utilizadas_gp

,tamanho,quantidade
0,G,6


In [96]:
df_estoque_gp = df_estoque_gp.merge(df_utilizadas_gp, on='tamanho', how='left', suffixes=('', '_utilizadas'))
df_estoque_gp['quantidade_utilizadas'] = df_estoque_gp['quantidade_utilizadas'].fillna(0)
df_estoque_gp['quantidade_utilizadas'] = df_estoque_gp['quantidade_utilizadas'].astype('int64')
df_estoque_gp

,tamanho,quantidade,quantidade_utilizadas
0,G,246,6
1,M,956,0
2,P,272,0
3,RN+,56,0


In [97]:
df_estoque_gp['qtd_restante'] = df_estoque_gp['quantidade'] - df_estoque_gp['quantidade_utilizadas']
df_estoque_gp

,tamanho,quantidade,quantidade_utilizadas,qtd_restante
0,G,246,6,240
1,M,956,0,956
2,P,272,0,272
3,RN+,56,0,56


In [109]:
try:
    print(str(df_estoque_gp.loc[df_estoque_gp['tamanho'] == 'XXG', 'qtd_restante'].iloc[0]))
except:
    print(0)

0
